In [ ]:
from functools import partial
import glob
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
import pybedtools as pbt
import re
import subprocess
import sys

sys.path.append("/dors/capra_lab/users/fongsl/tools/py_/")
sys.path.append("/dors/capra_lab/users/fongsl/tools/genome/")
import config_readwrite as crw
import chr_functions
import split_filename

In [3]:
name = "/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/config"
config, configfile_name = crw.read_config(name)

SHUF = config["SHUFFLES"]["shuf-all"]
REGIONS = config["CIS_TRANS"]["regions"]

CL = "GM12878"
FP_PATH = config["TF_FOOTPRINTING_JASPAR"][f"{CL}_bindetect"]
FP_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]["FP"]
ID_TAG = config["TF_FOOTPRINTING_JASPAR"]["ID_TAG"]

RE = config["TF_FOOTPRINTING"]["results"]

path, region_file, region = split_filename.split_filename(REGIONS)

0. string split to get TF name
1. intersect regions w footprint files
2. create matrix w/ region_id (should be 17605 regions x 693 archetypes) 

# functions

## step 0 

In [4]:
def get_arch_id(filename):
    """
    return archetype ID w/ chr/int split on first two values
    
    do string formatting
    """
    
    match = re.match(r"([a-z]+)([0-9]+)", filename.split("_")[0], re.I)  
    
    if match:
        items = match.groups()
    arch_id = ("".join(items))
    
    return arch_id

In [5]:
def get_tf_bound_file(f, filename, cell_line):

    tf_bound_file = os.path.join(f, "beds", f"{filename}_{cell_line}_1.8-filter_bound.bed")
    
    return tf_bound_file

In [6]:
def get_tf_paths(path, cell_line, id_tag):
    """
    return dictionary w/ key = archetype id and value = tuple of TF name and full file path.  
    
    1. get all the file names
    2. get arch_id name - string split function
    3. get TF name - str split
    4. get TF FP bound file
    5. make dictionary
    6. return dictionary
    """
    tf_dict ={}
    
    #1
    fs = glob.glob(os.path.join(path, f"*{id_tag}*"))
    print(len(fs))
    for f in fs:
        #2
        filename = os.path.split(f)[1]  # get the file name
        #3
        if "MA" in id_tag:
            arch_id = filename.upper()  
            tf_name = filename.split("_")[0]
        else:
            arch_id=get_arch_id(filename) #FOR ARCHTYPES ONLY
            
            #4
            tf_name = "_".join((filename.split(arch_id)[1]).split("_")[:-1])  # get TF name by splitting string. 

        
        #5
        tf_bound_file = get_tf_bound_file(f, filename, cell_line)  # get path to TF bound.bed file
        #6
        tf_dict[arch_id] = (tf_name, tf_bound_file)
    
    return tf_dict

## step 1 

In [7]:
def intersect_w_fp(regions, fp_name, fp_bed, outdir, cell_line, shuf):
    
    if shuf is True:
        cell_line = f"shuf-{cell_line}"
    
    print(outdir)
    if os.path.exists(os.path.join(outdir, cell_line)) is False:  # make a directory
        os.mkdir(os.path.join(outdir, cell_line))
        
    out = os.path.join(outdir, cell_line, f"{cell_line}-{fp_name}.bed")
    
    if os.path.exists(out) is False:  # if intersection hasn't already been done. 
        
        bedregions = pbt.BedTool(regions)
    
        bedregions.intersect(pbt.BedTool(fp_bed), wo = True).saveas(out)
        print(out)
    
    return out
    

In [8]:
def make_region_fp_matrix(outdir, cell_line, id_tag, shuf):

    if shuf is True:
        cell_line = f"shuf-{cell_line}"
        
    print(cell_line,"making matrix")
    """
    return dictionary of sample_id keys and pandas dataframes (values) for overlapping TF footprints
    
    1. get all region_x_TF.bed intersection files
    2. get sample_id 
    3. pick which columns and column names to load from file
    4. load file
    5. FILTER for overlaps w/ >5bp (size of TFBS)
    6. add to dictionary
    7. return dictionary
    """
    #1
    fs = glob.glob(os.path.join(outdir, cell_line, f"*{id_tag}*.bed"))
    
    dfs = {}
    for i, f in enumerate(fs):
        
        #2
        path, file, sample_id = split_filename.split_filename(f)
        #4
        usecols = list(np.arange(8))
        usecols.append(21)
        
        colnames = ["#chr", "start", "end", 'region_id', 
                    "#chr_fp", "start_fp", "end_fp", 
                    "fp_id", "overlap"]
        #4
        d = pd.read_csv(f, sep = '\t', header = None, usecols = usecols, names = colnames)
        #5
        d = d.loc[d["overlap"]>=5]
        #6
                
        arch_id = (sample_id.split(".bed")[0]).split("-")[-1]
        d[arch_id] = True
        #7
        dfs[sample_id] = d[["region_id", arch_id]].drop_duplicates()
        
        print(i)
    return dfs

## step 2

In [9]:
def join_fp_matrices(dfs, region_file):
    print("joining")
    
    """
    return pandas dataframe of all fp matrices
    1. open the region file
    2. consecutively join all other region_x_fp intersections
    3. fillin na w/ False
    
    """
    #1
    region_f = pd.read_csv(region_file, sep = '\t')
    
    #2
    for i, v in enumerate(dfs.values()):

        if i == 0:  # only merge the region file the first time
            regions = pd.merge(region_f, v, how = "left", on = "region_id")  
            
        else:
            regions = pd.merge(regions, v, how = "left")  #left merge the region results. 
            
    print(regions.shape)
    
    #3
    regions = regions.fillna(False)
    
    return regions

In [10]:
def check_regions_colnames(f):
    """
    rename columns for region file if needed. 
    """
    
    r = pd.read_csv(f, sep = '\t', nrows=2)  # read the first two lines to check. 
    
    if "region_id" not in list(r):  # if no column names:
        
        r = pd.read_csv(f, sep = '\t', header = None)  # read in dataframe
        r.columns = ["#chr", "start", "end", "region_id"]  # rename columns
    
        print(r.shape)

        r.to_csv(f, sep = '\t', index = False)  # resave dataframe w/ column names

# pipeline function 

In [11]:
def pipeline(outfile, cl, id_tag, regions, re, fp_path, shuf):
    
#if os.path.exists(outfile) is False:
    tfs = get_tf_paths(fp_path, cl, id_tag) ## get all the tf.bed paths

    for k, vs in tfs.items():  # per TF footprint file
        tfname, FP_bed = vs[0], vs[1]

        out = intersect_w_fp(regions, k, FP_bed, re, cl, shuf)  # intersect regions w fp

    dfs = make_region_fp_matrix(re, cl, id_tag, shuf)  # open the intersections as pandas dataframes, keep only FP overlaps > 5

    check_regions_colnames(regions)

    regions_df = join_fp_matrices(dfs, regions)

    regions_df.to_csv(outfile, sep = '\t', index = False)

#else:
    #print("done!")

# main

In [12]:
SHUFFLED = False
pipeline(FP_RE, CL, ID_TAG, REGIONS, RE, FP_PATH, SHUFFLED)

746
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_f

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footp

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footp

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footp

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footp

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footp

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# shuffle

In [13]:
CL_SHUF = f"shuf-{CL}"
FP_RE = os.path.join(RE, f"{CL_SHUF}_1.8-JASPAR_0.05-pval_morethan5bp.bed")
SHUFFLED = True
print(FP_RE)
pipeline(FP_RE, CL, ID_TAG, SHUF, RE, FP_PATH, SHUFFLED)

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878_1.8-JASPAR_0.05-pval_morethan5bp.bed
746
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-TGIF2LX_MA1571.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-POU6F2_MA0793.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-ESRRA_MA0592.3.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-HIC2_MA0738.1.bed
/data/ho

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-DUXA_MA0884.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-RXRAVDR_MA0074.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-RFX7_MA1554.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-FOSBJUNB_MA1135.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-DBP_MA0639.1.bed
/data/hodges_lab/A

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-FOXL2_MA1607.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-ISL2_MA0914.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-FOXH1_MA0479.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-BHLHE40_MA0464.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-HAND1TCF3_MA0092.1.bed
/data/hodges_l

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-FOS_MA0476.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-FOXA1_MA0148.4.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-MYCN_MA0104.4.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-HOXA1_MA1495.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-TCF3_MA0522.3.bed
/data/hodges_lab/ATAC-S

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-GMEB1_MA0615.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-POU2F2_MA0507.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-ZSCAN4_MA1155.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-OLIG2_MA0678.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-MAXMYC_MA0059.1.bed
/data/hodges_lab

...

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-SHOX_MA0630.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-ZSCAN29_MA1602.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-POU1F1_MA0784.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-POU5F1SOX2_MA0142.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-E2F7_MA0758.1.bed
/data/hodges_

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-PBX2_MA1113.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-HOXC8_MA1505.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-MEF2C_MA0497.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-BARX2_MA1471.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-NKX2-5_MA0063.2.bed
/data/hodges_lab/AT

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-HSF4_MA0771.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-MEOX2_MA0706.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-NFATC4_MA1525.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-ARNT2_MA1464.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-HOXA10_MA0899.1.bed
/data/hodges_lab/A

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-ZIC2_MA1629.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-EBF1_MA0154.4.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-PAX2_MA0067.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-CTCFL_MA1102.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-GM12878/shuf-GM12878-MAX_MA0058.3.bed
/data/hodges_lab/ATAC-ST

238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
